# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import holoviews as hv

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,81.95,69,93,1.01,US,1722708215
1,1,adamstown,-25.0660,-130.1015,69.35,72,6,13.73,PN,1722708216
2,2,port-aux-francais,-49.3500,70.2167,37.22,81,86,23.29,TF,1722708217
3,3,susuman,62.7833,148.1667,38.03,77,13,1.50,RU,1722708219
4,4,blackmans bay,-43.0167,147.3167,46.53,77,100,8.39,AU,1722708220


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng", "Lat",
    geo = True,
    tiles= "OSM",
    frame_width= 800,
    frame_height = 600,
    size="Humidity",
    color="City",
    alpha=0.75
)

# Display the map plot
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values

# Max temp lower than 27(C) or 80.6(F) but higher than 21(C) or 69.8(F), Zero Cloudiness, Wind Speed less than 4.5 (m/s)
perfect_cities = city_data_df.loc[(city_data_df["Max Temp"] <= 80.6) & (city_data_df["Max Temp"] >= 69.8) & 
                                  (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
perfect_cities = perfect_cities.dropna()

# Display sample data
perfect_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
221,221,ryotsu-minato,38.0833,138.4333,77.50,87,0,1.59,JP,1722708491
247,247,ushtobe,45.2514,77.9838,77.65,26,0,3.02,KZ,1722708522
274,274,guardo,42.7897,-4.8482,80.13,29,0,3.44,ES,1722708559
328,328,arnstadt,50.8333,10.9500,76.66,70,0,1.14,DE,1722708625
563,563,monteros,-27.1674,-65.4983,78.26,26,0,3.91,AR,1722708922


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = perfect_cities["City"]
country = perfect_cities["Country"]
lat = perfect_cities["Lat"]
lng = perfect_cities["Lng"]
humidity = perfect_cities["Humidity"]

hotel_df = perfect_cities[["City", "Country", "Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
221,ryotsu-minato,JP,38.0833,138.4333,87,
247,ushtobe,KZ,45.2514,77.9838,26,
274,guardo,ES,42.7897,-4.8482,29,
328,arnstadt,DE,50.8333,10.9500,70,
563,monteros,AR,-27.1674,-65.4983,26,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"acommodation.hotel","apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ryotsu-minato - nearest hotel: No hotel found
ushtobe - nearest hotel: No hotel found
guardo - nearest hotel: No hotel found
arnstadt - nearest hotel: No hotel found
monteros - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
221,ryotsu-minato,JP,38.0833,138.4333,87,No hotel found
247,ushtobe,KZ,45.2514,77.9838,26,No hotel found
274,guardo,ES,42.7897,-4.8482,29,No hotel found
328,arnstadt,DE,50.8333,10.9500,70,No hotel found
563,monteros,AR,-27.1674,-65.4983,26,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)